In [ ]:
# Import dependencies
import timeit
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
# Import Data
main_file_path = './data_customer.csv'
data = pd.read_csv(main_file_path)

print('Number of Samples: ', data.shape[0])
print('Number of Attributes: ', data.shape[1]-1)
data.head()

### Data Preprocessing

In [ ]:
# Display Data types of each column
data.dtypes

In [ ]:
# Variable TotalCharges should be a continuous numeric data type rather than a discrete object data type
data.TotalCharges = pd.to_numeric(data.TotalCharges, errors='coerce')

# Check for NULL entries
data.isnull().sum()

In [ ]:
# Remove all entries that contain NULL
data.dropna(inplace=True)

# Remove customer ID
df = data.iloc[:,1:]

# Separate X and Y
ydata = df.Churn #df_upsampled.Churn
xdata_raw = df.drop(['Churn'], axis=1) #df_upsampled.drop(['Churn'], axis=1)

# Convert Chrun into binary numeric variable
ydata.replace(to_replace='Yes', value=1, inplace=True)
ydata.replace(to_replace='No', value=0, inplace=True)

# Convert all categorial variables in xdata into dummy variables
xdata = pd.get_dummies(xdata_raw)



In [ ]:
i_class0 = np.where(ydata == 0)[0]
i_class1 = np.where(ydata == 1)[0]
print("If choose 0 for all guesses: ", len(i_class0)/data.shape[0])

## Data Visualization

In [ ]:
import seaborn as sns

ax = sns.countplot(ydata,label="Count")       
Not_Churned, Churned = ydata.value_counts()
print('Number Churned: ', Churned)
print('Number Not Churned: ',Not_Churned)

In [ ]:
#Import train_test_split
from sklearn.model_selection import train_test_split

# Split data into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(xdata, ydata, test_size=.2, random_state=101, stratify=ydata)

In [ ]:
# Scale all features from 0 to 1
from sklearn.preprocessing import MinMaxScaler

features = X_train.columns.values
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_train.columns = features
X_test = pd.DataFrame(scaler.transform(X_test))
X_test.columns = features

X_train.head()


### Predictive Models - Parameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

def grid_search(estimator, param_grid, X_train, y_train):
    grid = GridSearchCV(estimator, param_grid, cv=10, scoring='accuracy', return_train_score=True)
    grid.fit(X_train, y_train)
#     print(pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']])

#     print()
    # examine the best model
    print('Best_score = ', grid.best_score_)
    print('Best_params = ', grid.best_params_)
    return grid

##### **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

param_grid={
             'criterion': ["gini", "entropy"],
             'max_depth': range(2,21,2),
           }

dtree_grid = grid_search(DecisionTreeClassifier(), param_grid, X_train, y_train)
joblib.dump(dtree_grid, 'dtree_grid.pkl')


##### **Nerual Network**

In [ ]:
from sklearn.neural_network import MLPClassifier

param_grid = {
               'solver': ['lbfgs','sgd','adam'],
               'alpha': 10.0 ** -np.arange(1, 5),
               'hidden_layer_sizes':[1,2,5,10], 
               'random_state':[0,4,8]
             }

nn_grid = grid_search(MLPClassifier(), param_grid, X_train, y_train)
joblib.dump(nn_grid, 'nn_grid.pkl')


##### **Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
               'loss': ['deviance', 'exponential'],    
               'learning_rate': [0.05, 0.1, 0.2],
               'max_depth': [2,5,10,15,20],
               'min_samples_split': [2,5,10,15]
             }

gb_grid = grid_search(GradientBoostingClassifier(random_state = 0), param_grid, X_train, y_train)
joblib.dump(gb_grid, 'gb_grid.pkl')
 

##### **Support Vector Machine**

In [ ]:
from sklearn.svm import SVC

param_grid = [
               {
                 'C': [1, 10, 100, 1000], 
                 'kernel': ['linear']
               },
               {
                 'C': [1, 10, 100, 1000], 
                 'gamma': [0.001, 0.0001], 
                 'kernel': ['rbf']
               },
             ]

svc_grid = grid_search(SVC(), param_grid, X_train, y_train)
joblib.dump(svc_grid, 'svc_grid.pkl')

##### **K Nearest Neighbors**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
               'n_neighbors': range(1, 31),    
               'weights': ['uniform', 'distance']
             }

knn_grid = grid_search(KNeighborsClassifier(), param_grid, X_train, y_train)
joblib.dump(knn_grid, 'knn_grid.pkl')

### Predictive Models - Learning Curves

In [ ]:
#Load trained models
dtree_grid = joblib.load('dtree_grid.pkl') 
nn_grid = joblib.load('nn_grid.pkl') 
gb_grid = joblib.load('gb_grid.pkl') 
svc_grid = joblib.load('svc_grid.pkl') 
knn_grid = joblib.load('knn_grid.pkl') 

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, estimator_name, X, y, ylim=None, cv=10, train_sizes=[19,50,100,200,300,400]):
    plt.figure()
    param_str = ', '.join('\'{}\': {}'.format(key, val) for key, val in estimator.best_params_.items())
    plt.title(estimator_name + ' ' + "Learning Curves")
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator.best_estimator_, X, y, cv=cv, scoring='accuracy', train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.savefig('Learning_curve-{}.png'.format(estimator_name))
    plt.show()
    return plt

In [ ]:
#Decision Tree
plot_learning_curve(dtree_grid, 'Decision_Tree', X_train, y_train, (0.7,1.01), train_sizes=[19,50,100,220,300,409])

In [ ]:
#Nueral Network
plot_learning_curve(nn_grid, 'Neural_Network', X_train, y_train, (0.7,1.01), cv=10)

In [ ]:
#Boosting
plot_learning_curve(gb_grid, 'Boosting', X_train, y_train, (0.7,1.01), cv=10, train_sizes=[60,100,200,300,400])

In [ ]:
#SVM
plot_learning_curve(svc_grid, 'Support_Vector_Machine', X_train, y_train, (0.7,1.01), cv=10, train_sizes=[60,100,200,300,400])

In [ ]:
#KNN
plot_learning_curve(knn_grid, 'K-Nearest_Neighbors', X_train, y_train, (0.7,1.01), cv=10, train_sizes=[24,50,100,220,300,409])

## Complexity Curves - Hyperparameters

In [ ]:
from sklearn.model_selection import validation_curve
def complexity_curve(estimator, title, param_name, param_range, X=X_train,y=y_train):
    train_scores, test_scores = validation_curve(estimator.best_estimator_, X, y, param_name=param_name, param_range=param_range,cv=10, scoring="accuracy")
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.title("Complexity Curve with " + title)
    plt.xlabel(param_name)
    plt.ylabel("Accuracy Score")
    plt.ylim(0.6, 1.05)

    plt.plot(param_range, train_scores_mean,'o-', label="Training score",
                 color="r")
    plt.fill_between(param_range, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.plot(param_range, test_scores_mean,'o-', label="Cross-validation score",
                 color="g")
    plt.fill_between(param_range, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,
                     color="g")
    plt.legend(loc="lower right")
    plt.savefig('Complexity_curve-{}.png'.format(title))
    plt.show()

In [ ]:
# Decision Tree
complexity_curve(dtree_grid, 'Decsion_tree', 'max_depth', [1,2,3,4,5,7,8,9,10,11])

In [ ]:
# Neural Network
complexity_curve(nn_grid, 'Neural_Network', 'hidden_layer_sizes', [1,3,6,10,13,15])

In [ ]:
# Boosting
complexity_curve(gb_grid, 'Boosting', 'max_depth', [1,2,4,6,8,10,12,16,20])

In [ ]:
# SVM
complexity_curve(svc_grid, 'Support_Vector_Machine', 'gamma', [.000001,.00001,.0001,.001,.002])

In [ ]:
# KNN
complexity_curve(knn_grid, 'K-Nearest_Neighbors', 'n_neighbors', [1,5,10,15,19,25,30])

## Learning Curves - Iterations

In [ ]:
from sklearn.metrics import accuracy_score

title = 'Neural_Network'
iterations= [1,10,25,50,100,125]
fit_times = []
pred_times = []
train_scores = []
test_scores = []

for iteration in iterations: 
    estimator = MLPClassifier(random_state=6,hidden_layer_sizes=10,solver='adam',alpha=0.1, max_iter=iteration)
    start_time = timeit.default_timer()
    estimator.fit(X_train, y_train) 
    fit_times.append(timeit.default_timer() - start_time)

    start_time1 = timeit.default_timer()
    y_pred = estimator.predict(X_test)
    pred_times.append(timeit.default_timer() - start_time1)
    test_scores.append(accuracy_score(y_test, y_pred))

    y_pred = estimator.predict(X_train)
    train_scores.append(accuracy_score(y_train, y_pred))


plt.figure(0)
plt.grid()
plt.plot(iterations, train_scores,'o-', label="Training score", color="r")
plt.plot(iterations, test_scores,'o-', label="Testing score", color="g")
plt.ylabel('Accuracy Score')
plt.xlabel('Iterations')
plt.title(title + ' Interation Learning Curve')
plt.legend(loc="best")
plt.savefig('Iteration_Score_curve-{}.png'.format(title))
plt.show()

plt.figure(1)
plt.grid()
plt.plot(iterations, fit_times,'o-', label="Fit Time", color="b")
plt.plot(iterations, pred_times,'o-', label="Prediction Time", color="g")
plt.ylabel('Time (s)')
plt.xlabel('Iterations')
plt.title(title + ' Interation Learning Curve - Fit and Prediction Times')
plt.legend(loc="best")
plt.savefig('Iteration_Time_curve-{}.png'.format(title))
plt.show()

In [ ]:
# Boosting
title = 'Boosting'
iterations= [1,10,25,50,100,550]
fit_times = []
pred_times = []
train_scores = []
test_scores = []

for iteration in iterations: 
    estimator = GradientBoostingClassifier(min_samples_split=15, loss='deviance', learning_rate=0.2, max_depth=4, n_estimators=iteration)
    start_time = timeit.default_timer()
    estimator.fit(X_train, y_train) 
    fit_times.append(timeit.default_timer() - start_time)

    start_time1 = timeit.default_timer()
    y_pred = estimator.predict(X_test)
    pred_times.append(timeit.default_timer() - start_time1)
    test_scores.append(accuracy_score(y_test, y_pred))

    y_pred = estimator.predict(X_train)
    train_scores.append(accuracy_score(y_train, y_pred))


plt.figure(0)
plt.grid()
plt.plot(iterations, train_scores,'o-', label="Training score", color="r")
plt.plot(iterations, test_scores,'o-', label="Testing score", color="g")
plt.ylabel('Accuracy Score')
plt.xlabel('Iterations')
plt.title(title + ' Interation Learning Curve')
plt.legend(loc="best")
plt.savefig('Iteration_Score_curve-{}.png'.format(title))
plt.show()

plt.figure(1)
plt.grid()
plt.plot(iterations, fit_times,'o-', label="Fit Time", color="b")
plt.plot(iterations, pred_times,'o-', label="Prediction Time", color="g")
plt.ylabel('Time (s)')
plt.xlabel('Iterations')
plt.title(title + ' Interation Learning Curve - Fit and Prediction Times')
plt.legend(loc="best")
plt.savefig('Iteration_Time_curve-{}.png'.format(title))
plt.show()


In [ ]:
# SVM
title = 'Support_Vector_Machine'
iterations= [1,20,70,100]
fit_times = []
pred_times = []
train_scores = []
test_scores = []

for iteration in iterations: 
    estimator = SVC(C=1000,gamma=0.001,kernel='rbf', max_iter=iteration)
    start_time = timeit.default_timer()
    estimator.fit(X_train, y_train) 
    fit_times.append(timeit.default_timer() - start_time)

    start_time1 = timeit.default_timer()
    y_pred = estimator.predict(X_test)
    pred_times.append(timeit.default_timer() - start_time1)
    test_scores.append(accuracy_score(y_test, y_pred))

    y_pred = estimator.predict(X_train)
    train_scores.append(accuracy_score(y_train, y_pred))


plt.figure(0)
plt.grid()
plt.plot(iterations, train_scores,'o-', label="Training score", color="r")
plt.plot(iterations, test_scores,'o-', label="Testing score", color="g")
plt.ylabel('Accuracy Score')
plt.xlabel('Iterations')
plt.title(title + ' Interation Learning Curve')
plt.legend(loc="best")
plt.savefig('Iteration_Score_curve-{}.png'.format(title))
plt.show()

plt.figure(1)
plt.grid()
plt.plot(iterations, fit_times,'o-', label="Fit Time", color="b")
plt.plot(iterations, pred_times,'o-', label="Prediction Time", color="g")
plt.ylabel('Time (s)')
plt.xlabel('Iterations')
plt.title(title + ' Interation Learning Curve - Fit and Prediction Times')
plt.legend(loc="best")
plt.savefig('Iteration_Time_curve-{}.png'.format(title))
plt.show()

In [ ]:
def final_test_score(estimator, title):
    start_time = timeit.default_timer()
    estimator.fit(X_train, y_train) 
    print(title + ' Fit Time: ',round(timeit.default_timer() - start_time,5))

    start_time1 = timeit.default_timer()
    y_pred = estimator.predict(X_test)
    print(title + ' Prediction Time: ',round(timeit.default_timer() - start_time1,5))
    print(title + ' Final Test Score: ',round(accuracy_score(y_test, y_pred)*100,1),'%')

#### - Decision Tree

In [ ]:
final_test_score(dtree_grid.best_estimator_, 'Decision_Tree')


#### - Neural Network

In [ ]:
final_test_score(nn_grid.best_estimator_, 'Neural_Network')

#### - Boosting

In [ ]:
final_test_score(gb_grid.best_estimator_, 'Boosting')

#### - Support Vector Machine

In [ ]:
final_test_score(svc_grid.best_estimator_, 'Support_Vector_Machine')

#### -  K-NN

In [ ]:
final_test_score(knn_grid.best_estimator_, 'K-Nearest_Neighbors')